# Rotina para Acompanhamento Macroeconômico

## Importando as bibliotecas utilizadas

In [1]:
import pandas as pd
import requests

## Função de extração das tabelas por url

In [2]:
def retorna_tabelas(url: str) -> list:
    try:
        url = url

        header = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        }

        r = requests.get(url, headers = header)
        tabelas = pd.read_html(r.text)
        return tabelas
    
    except HTTPError as e:
        print("The server returned an HTTP error")
    
    except URLError as e:
        print("The server could not be found!")

## Commodities

In [3]:
def tabela_commodities(tabela_commodities: list, df: pd.DataFrame) -> pd.DataFrame:
    for i in range(0, 5):
        commodities = tabela_commodities[i].dropna(how = 'all', axis = 1)
        commodities = commodities.rename(columns = {'Unnamed: 3' : 'Var', 
                                                    f'{commodities.columns[0]}' : 'Commodities'})
        commodities = commodities.set_index(commodities.columns[0])
        df = pd.concat([df, commodities])
    
    df.rename_axis('Commodities', inplace = True)
    
    return df

### Preços Atuais

In [4]:
url_commodities_atuais = 'https://tradingeconomics.com/commodities'
tabela_commodities_atuais = retorna_tabelas(url_commodities_atuais)

In [5]:
commodities_atuais = pd.DataFrame(columns = ['Price', 'Var', 'Day', 'Weekly', 'Monthly', 'YTD', 'Date'])
commodities_atuais = tabela_commodities(tabela_commodities_atuais, commodities_atuais)

In [6]:
commodities_atuais

,Price,Var,Day,Weekly,Monthly,YTD,Date
Commodities,,,,,,,
Crude Oil USD/Bbl,61.4500,2.29,3.87%,4.94%,2.85%,26.65%,Apr/01
Brent USD/Bbl,64.8600,2.12,3.38%,4.70%,3.45%,25.21%,Apr/01
Natural gas USD/MMBtu,2.6390,0.03,1.19%,2.68%,-7.04%,3.94%,Apr/01
Gasoline USD/Gal,2.0223,0.06,3.19%,5.28%,4.44%,43.42%,Apr/01
Heating oil USD/Gal,1.8316,0.06,3.49%,4.79%,1.30%,23.42%,Apr/01
Ethanol USD/Gal,1.9100,0.03,1.33%,6.11%,7.61%,33.29%,Apr/01
Naphtha USD/T,564.3200,8.62,-1.51%,-0.96%,0.60%,30.42%,Apr/01
Propane USD/Gal,0.9500,0.02,2.51%,3.11%,-2.33%,46.69%,Apr/01
Uranium USD/Lbs,31.2500,0.00,0.00%,3.82%,12.21%,1.79%,Apr/01


#### Salvando em excel

In [7]:
commodities_atuais.to_excel('Commodities_Atuais.xls', encoding = 'UTF-8')

### Preços Futuros

In [8]:
url_commodities_futuras = 'https://tradingeconomics.com/forecast/commodity'
tabela_commodities_futuras = retorna_tabelas(url_commodities_futuras)

In [9]:
commodities_futuras = pd.DataFrame(columns = ['Price', 'Var', 'Day'])
commodities_futuras = tabela_commodities(tabela_commodities_futuras, commodities_futuras)

In [10]:
commodities_futuras

,Price,Var,Day,Q2/21,Q3/21,Q4/21,Q1/22
Commodities,,,,,,,
Crude Oil,61.4500,2.29,3.87%,58.4390,55.5754,52.8531,50.2600
Brent,64.8600,2.12,3.38%,61.8051,58.8929,56.1234,53.4771
Natural gas,2.6390,0.03,1.19%,2.4904,2.3503,2.2178,2.0930
Gasoline,2.0223,0.06,3.19%,1.9086,1.8015,1.7001,1.6047
Heating oil,1.8316,0.06,3.49%,1.7523,1.6765,1.6037,1.5343
Ethanol,1.9100,0.03,1.33%,1.7889,1.6755,1.5693,1.4697
Naphtha,564.3200,8.62,-1.51%,536.3900,509.8700,484.5800,460.6000
Propane,0.9500,0.02,2.51%,0.8800,0.8300,0.7700,0.7200
Uranium,31.2500,0.00,0.00%,30.7313,30.2219,29.7188,29.2250


#### Salvando em excel

In [11]:
commodities_futuras.to_excel('Commodities_Futuras.xls', encoding = 'UTF-8')

## Índices Macroeconômicos

In [12]:
url_indices_macro = 'https://tradingeconomics.com/'
indices_macro = retorna_tabelas(url_indices_macro)

In [13]:
indices_macro = indices_macro[-1]
indices_macro = indices_macro.rename(columns = {indices_macro.columns[0] : 'Países'})
indices_macro = indices_macro.set_index('Países')

In [14]:
indices_macro

,GDP,GDP YoY,GDP QoQ,Interest rate,Inflation rate,Jobless rate,Gov. Budget,Debt/GDP,Current Account,Population
Países,,,,,,,,,,
United States,21433,-2.40%,4.30%,0.25%,1.70%,6.00%,-4.60%,107.60%,-3.10,329.48
China,14343,6.50%,2.60%,3.85%,-0.20%,5.50%,-3.70%,52.60%,1.00,1400.05
Euro Area,13336,-4.90%,-0.70%,0.00%,1.30%,8.10%,-0.60%,77.60%,2.30,341.90
Japan,5082,-1.40%,2.80%,-0.10%,-0.40%,2.90%,-2.40%,236.60%,3.20,126.01
Germany,3846,-3.70%,0.30%,0.00%,1.70%,4.50%,-4.80%,59.80%,7.10,83.20
India,2869,0.40%,7.90%,4.00%,5.03%,6.50%,-9.50%,69.62%,-0.90,1312.24
United Kingdom,2829,-7.30%,1.30%,0.10%,0.40%,5.00%,-16.90%,100.20%,-3.80,66.65
France,2716,-4.90%,-1.40%,0.00%,1.10%,8.00%,-9.20%,115.70%,-0.70,66.98
Italy,2001,-6.60%,-1.90%,0.00%,0.80%,9.00%,-9.50%,155.60%,3.00,60.36


#### Salvando em excel

In [15]:
indices_macro.to_excel('Índices_Macro.xls', encoding = 'UTF-8')

## Índices Bolsas Mundiais

In [16]:
def trata_indices(df: pd.DataFrame) -> pd.DataFrame:
    df.index = df.index.map(lambda x: x[:-len('derived')] if 'derived' in x else x)
    return df

### Atuais

In [17]:
url_indices_atuais = 'https://www.investing.com/indices/major-indices'
tabelas_indices_atuais = retorna_tabelas(url_indices_atuais)

In [18]:
indices_mundo_atuais = tabelas_indices_atuais[0][['Index', 'Last', 'High', 'Low', 'Chg.', 'Chg. %']].set_index('Index')
indices_mundo_atuais = trata_indices(indices_mundo_atuais)
indices_mundo_atuais.head()

,Last,High,Low,Chg.,Chg. %
Index,,,,,
Dow Jones,33153.21,33167.17,32985.35,171.66,+0.52%
S&P 500,4019.87,4020.63,3992.78,46.98,+1.18%
Nasdaq,13480.11,13487.08,13404.18,233.24,+1.76%
Small Cap 2000,2248.22,2252.16,2225.29,0.00,0.00%
S&P 500 VIX,17.33,18.64,17.29,0.00,0.00%


#### Salvando em excel

In [19]:
indices_mundo_atuais.to_excel('Indices_Mundo_Atuais.xls', encoding = 'UTF-8')

### Futuros

In [20]:
url_indices_futuros = 'https://br.investing.com/indices/indices-futures'
tabelas_indices_futuros = retorna_tabelas(url_indices_futuros)

In [21]:
indices_mundo_futuros = tabelas_indices_futuros[0][['Índice', 'Mês', 'Último', 
                                                   'Máxima', 'Mínima', 'Var. %', 'Hora']].set_index('Índice')
indices_mundo_futuros = trata_indices(indices_mundo_futuros)
indices_mundo_futuros.head()

,Mês,Último,Máxima,Mínima,Var. %,Hora
Índice,,,,,,
Ibovespa,Ex.Abr 21,114.79200,117.50000,114.70200,"-1,88%",01/04
IBRX50,Ex.Jun 21,19.25100,19.55100,19.25100,"-1,00%",01/04
US 30,Ex.Jun 21,33.18950,33.26200,33.01650,"+0,46%",10:14:57
S&P 500,Ex.Jun 21,4.02712,4.03762,4.01262,"+0,43%",10:14:58
Nasdaq,Ex.Jun 21,13.34700,13.38962,13.32575,"+0,23%",10:14:58


#### Salvando em excel

In [22]:
indices_mundo_futuros.to_excel('Indices_Mundo_Futuros.xls', encoding = 'UTF-8')